In [1]:
from pathlib import Path
from pyraphtory.context import PyRaphtory

In [2]:
pr = PyRaphtory(spout_input=Path('/tmp/lotr.csv'), builder_script=Path('builder.py'),  builder_class='LotrGraphBuilder', mode='batch', logging=True).open()

b'WARNING: sun.reflect.Reflection.getCallerClass is not supported. This will impact performance.\n'
b"16:12:18.414 [io-compute-42] INFO  com.raphtory.spouts.FileSpout - Spout: Processing file 'lotr.csv' ...\n"
b'16:12:18.492 [io-compute-42] INFO  com.raphtory.internals.management.Prometheus$ - Prometheus started on port /0:0:0:0:0:0:0:0:9999\n'
b'16:12:18.935 [spawner-akka.actor.default-dispatcher-3] INFO  akka.event.slf4j.Slf4jLogger - Slf4jLogger started\n'
b"16:12:19.088 [io-compute-42] INFO  com.raphtory.internals.management.PartitionsManager$ - Creating '1' Partition Managers for raphtory_1349842851.\n"
b'16:12:19.155 [io-compute-42] INFO  com.raphtory.internals.management.Py4JServer - Starting PythonGatewayServer...\n'
b'16:12:19.156 [io-compute-42] INFO  com.raphtory.internals.management.Py4JServer - Started PythonGatewayServer on port 41785 host: localhost/127.0.0.1\n'
b'16:12:19.159 [io-compute-42] INFO  com.raphtory.internals.management.Py4JServer - Writing PythonGatewayServe

In [3]:
from pyraphtory.algo import Vertex, Iterate, Step

In [4]:
PR_LABEL = 'prlabel'

class PGStep1(Step):
    def eval(self, v: Vertex):
        initLabel = 1.0
        v[PR_LABEL] = initLabel
        out_degree = v.out_degree()
        if out_degree > 0:
            msg = initLabel / out_degree
            v.message_outgoing_neighbours(msg)

class PGIterate1(Iterate):
    def __init__(self, iterations: int, execute_messaged_only: bool, damping_factor: float = 0.85):
        super().__init__(iterations, execute_messaged_only)
        self.damping_factor = damping_factor

    def eval(self, v: Vertex):
        current_label = v[PR_LABEL]
        queue = v.message_queue()
        summed_queue = sum(queue)
        new_label = (1 - self.damping_factor) + self.damping_factor * summed_queue
        v[PR_LABEL] = new_label

        out_degree = v.out_degree()

        if out_degree > 0:
            v.message_outgoing_neighbours(new_label / out_degree)

        if abs(new_label - current_label) < 0.00001:
            v.vote_to_halt()


In [5]:
rg = pr.graph()

In [ ]:
local_sink = pr.local_sink()

In [ ]:
tracker = rg.at(32674) \
                .past() \
                .step(PGStep1()) \
                .iterate(PGIterate1(iterations=100, execute_messaged_only=False)) \
                .select([PR_LABEL]) \
                .write_to(local_sink)

In [ ]:
print(tracker)

In [ ]:
tracker.wait_for_job()

In [ ]:
res = local_sink.results()

In [ ]:
res[0:5]